In [1]:
import pandas as pd
import json 
import numpy as np 
import re
import sqlite3

In [3]:
df_datas = pd.read_json('data/airlines-1558527599826.json', lines=True)
df_datas

created_at            id        id_str  \
0     2019-05-22 12:20:00+00:00  1.131173e+18  1.131173e+18   
1     2019-05-22 12:20:01+00:00  1.131173e+18  1.131173e+18   
2     2019-05-22 12:20:02+00:00  1.131173e+18  1.131173e+18   
3     2019-05-22 12:20:12+00:00  1.131173e+18  1.131173e+18   
4     2019-05-22 12:20:28+00:00  1.131173e+18  1.131173e+18   
...                         ...           ...           ...   
14384 2019-05-23 11:42:26+00:00  1.131526e+18  1.131526e+18   
14385 2019-05-23 11:42:27+00:00  1.131526e+18  1.131526e+18   
14386 2019-05-23 11:42:27+00:00  1.131526e+18  1.131526e+18   
14387 2019-05-23 11:42:35+00:00  1.131526e+18  1.131526e+18   
14388 2019-05-23 11:42:44+00:00  1.131526e+18  1.131526e+18   

                                                    text display_text_range  \
0      La ruta de easyJet entre Londres y Menorca tra...           [0, 113]   
1      RT @bttr_as1: @goody_tracy Here’s a list of so...                NaN   
2                                       @British_Airways                NaN   
3      RT @TheRaceRadio: Nice change by @AmericanAir....                NaN   
4      RT @sandeeprrao1991: BREAKING:-\nKLM to fly 3x...                NaN   
...                                                  ...                ...   
14384     @yvettetravel were you invited to attend this?                NaN   
14385  RT @RoyalFamily: The Queen sees how the @Briti...                NaN   
14386  @RoyalSpectator @British_Airways She looks AMA...           [33, 60]   
14387  @maj_fox @British_Airways Should happen more o...           [26, 94]   
14388  @British_Airways delayed 5hrs on flight BA0103...                NaN   

                                                  source  truncated  \
0      <a href="https://about.twitter.com/products/tw...        0.0   
1      <a href="http://twitter.com/download/iphone" r...        0.0   
2      <a href="http://twitter.com/download/android" ...        0.0   
3      <a href="http://twitter.com/download/android" ...        0.0   
4      <a href="https://mobile.twitter.com" rel="nofo...        0.0   
...                                                  ...        ...   
14384  <a href="http://twitter.com/download/iphone" r...        0.0   
14385  <a href="https://mobile.twitter.com" rel="nofo...        0.0   
14386  <a href="http://twitter.com" rel="nofollow">Tw...        0.0   
14387  <a href="http://twitter.com/download/android" ...        0.0   
14388  <a href="http://twitter.com/download/iphone" r...        1.0   

       in_reply_to_status_id  in_reply_to_status_id_str  in_reply_to_user_id  \
0                        NaN                        NaN                  NaN   
1                        NaN                        NaN                  NaN   
2               1.131033e+18               1.131033e+18         3.943766e+08   
3                        NaN                        NaN                  NaN   
4                        NaN                        NaN                  NaN   
...                      ...                        ...                  ...   
14384                    NaN                        NaN         3.046262e+08   
14385                    NaN                        NaN                  NaN   
14386           1.131512e+18               1.131512e+18         1.590887e+09   
14387           1.131484e+18               1.131484e+18         1.085556e+18   
14388                    NaN                        NaN         1.833219e+07   

       ...  filter_level lang            timestamp_ms  \
0      ...           low   es 2019-05-22 12:20:00.406   
1      ...           low   en 2019-05-22 12:20:01.645   
2      ...           low  und 2019-05-22 12:20:02.560   
3      ...           low   en 2019-05-22 12:20:12.449   
4      ...           low   en 2019-05-22 12:20:28.237   
...    ...           ...  ...                     ...   
14384  ...           low   en 2019-05-23 11:42:26.066   
14385  ...           low   en 2019-05-23 11:4

In [8]:
df_datas = df_datas.dropna(subset=['user'])

def cleaner(text):
    '''Remove URLs'''
    text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
    return text


def punc(text1):
    '''Remove punctuations and symbols except ! & ?'''
    punct = '''()-[]{};:'"\,<>./@#$%^&*_~'''
    for word in text1:
        if word in punct:
            text1 = text1.replace(word, "")
    return text1

def to_lower(text2):
    '''Convert string to lower case'''
    text2 = text2.lower()
    return text2

#def emoji(text3):
    '''Convert emoji to text'''
    
def abbre(text4):
    '''Expand abbreviations, common ones'''
    abbre_dict = {"asap": "as soon as possible", "atm": "at the moment", "fyi": "for your information", "np": "no problem",
                 "pls": "please", "thx": "thanks", "lmk": "let me know", "lol": "laugh out loud", "nvm": "nevermind", 
                 "tba": "to be honest", "tbd": "to be decided"}
    for key, value in abbre_dict.items():
        text4 = text4.replace(key,value)
    return text4
    
    
#remove non-English tweets
df_datas = df_datas.loc[df_datas['lang'] =='en']
for i in range(len(df_datas['text'].values)):
    text = df_datas['text'].iloc[i]
    lang = df_datas['lang'].iloc[i]
    df_datas['text'].iloc[i] = cleaner(text)
    df_datas['text'].iloc[i] = punc(text)
    df_datas['text'].iloc[i] = to_lower(text)
    df_datas['text'].iloc[i] = abbre(text)
    
    
df_datas

C:\Users\12697\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


created_at            id        id_str  \
1     2019-05-22 12:20:01+00:00  1.131173e+18  1.131173e+18   
3     2019-05-22 12:20:12+00:00  1.131173e+18  1.131173e+18   
4     2019-05-22 12:20:28+00:00  1.131173e+18  1.131173e+18   
5     2019-05-22 12:20:36+00:00  1.131173e+18  1.131173e+18   
6     2019-05-22 12:20:36+00:00  1.131173e+18  1.131173e+18   
...                         ...           ...           ...   
14384 2019-05-23 11:42:26+00:00  1.131526e+18  1.131526e+18   
14385 2019-05-23 11:42:27+00:00  1.131526e+18  1.131526e+18   
14386 2019-05-23 11:42:27+00:00  1.131526e+18  1.131526e+18   
14387 2019-05-23 11:42:35+00:00  1.131526e+18  1.131526e+18   
14388 2019-05-23 11:42:44+00:00  1.131526e+18  1.131526e+18   

                                                    text display_text_range  \
1      rt   here’s a list of some of  clients. they s...                NaN   
3      rt  nice change by  bikes now pay standard che...                NaN   
4      rt  breaking:-\nklm to fly 3x weekly btw  and ...                NaN   
5      easyjet was on a slippery slope when it charge...                NaN   
6      thanks  i really needed the extra 2 hour delay...                NaN   
...                                                  ...                ...   
14384                   were you invited to attend this?                NaN   
14385  rt  the queen sees how the  uniform has evolve...                NaN   
14386                        she looks amazing today!! 😍           [33, 60]   
14387    should happen more often for our service men...           [26, 94]   
14388   delayed 5hrs on flight ba0103 from lhr to cal...                NaN   

                                                  source  truncated  \
1      <a href="http://twitter.com/download/iphone" r...        0.0   
3      <a href="http://twitter.com/download/android" ...        0.0   
4      <a href="https://mobile.twitter.com" rel="nofo...        0.0   
5      <a href="http://twitter.com" rel="nofollow">Tw...        0.0   
6      <a href="http://twitter.com/download/iphone" r...        0.0   
...                                                  ...        ...   
14384  <a href="http://twitter.com/download/iphone" r...        0.0   
14385  <a href="https://mobile.twitter.com" rel="nofo...        0.0   
14386  <a href="http://twitter.com" rel="nofollow">Tw...        0.0   
14387  <a href="http://twitter.com/download/android" ...        0.0   
14388  <a href="http://twitter.com/download/iphone" r...        1.0   

       in_reply_to_status_id  in_reply_to_status_id_str  in_reply_to_user_id  \
1                        NaN                        NaN                  NaN   
3                        NaN                        NaN                  NaN   
4                        NaN                        NaN                  NaN   
5                        NaN                        NaN                  NaN   
6                        NaN                        NaN                  NaN   
...                      ...                        ...                  ...   
14384                    NaN                        NaN         3.046262e+08   
14385                    NaN                        NaN                  NaN   
14386           1.131512e+18               1.131512e+18         1.590887e+09   
14387           1.131484e+18               1.131484e+18         1.085556e+18   
14388                    NaN                        NaN         1.833219e+07   

       ...  filter_level lang            timestamp_ms  \
1      ...           low   en 2019-05-22 12:20:01.645   
3      ...           low   en 2019-05-22 12:20:12.449   
4      ...           low   en 2019-05-22 12:20:28.237   
5      ...           low   en 2019-05-22 12:20:36.475   
6      ...           low   en 2019-05-22 12:20:36.540   
...    ...           ...  ...                     ...   
14384  ...           low   en 2019-05-23 11:42:26.066   
14385  ...           low   en 2019-05-23 11:4